In [1]:
import numpy as np
import spatialmath as sm
import quaternion
import hrr_common
import rospy
from geometry_msgs.msg import PoseStamped
def get_SE3_from_pose_stamped(msg):
    p = hrr_common.vec32np(msg.pose.position)
    q = hrr_common.quat2np(msg.pose.orientation)
    return hrr_common.posquat2homog(p, q)

### Vision input

In [ ]:
cobot.T_B_E_robot

In [44]:
obj_pos_guess = np.r_[0.50 ,0, 0.45]
obj_orient_guess = np.quaternion(0.20924,-0.071122,-0.72064, 0.657136) #w x y z
obj_orient_guess 

quaternion(0.20924, -0.071122, -0.72064, 0.657136)

In [49]:
a_test = np.zeros([3,3])
a_test[:,0] = np.r_[0,0,1]
a_test[:,1] = np.r_[1,0,0]
a_test[:,2] = np.cross(a_test[:,0], a_test[:,1])
a_test

array([[0., 1., 0.],
       [0., 0., 1.],
       [1., 0., 0.]])

In [ ]:
a_test = np.zeros([3,3])
a_test[:,0] = np.r_[0,0,1]
a_test[:,2] = -np.r_[0,1,0]
a_test[:,1] = np.cross(a_test[:,2], a_test[:,0])
a_test

In [39]:
obj_orient_guess =  quaternion.from_rotation_matrix(a_test)
obj_orient_guess 

quaternion(-0.5, 0.5, 0.5, 0.5)

### Hardcode precise vision input

In [ ]:
a_test

In [ ]:
#Get the rotation matrix for the object: x axis points up to z
a = sm.SE3()
a = a @ sm.SE3.Ry(np.deg2rad(-90))
a.t[0:3] = obj_pos_guess
a

In [ ]:
#If wanted, turn the object 90 degrees (this will affect last link of robot, turned by 90 degrees)
a = a @ sm.SE3.Rx(np.deg2rad(90))

In [ ]:
a

In [ ]:
np.zeros([3,3])

In [ ]:
a.R[:,0]

In [ ]:
#Save as quaternion, this is what vision needs to give us
obj_orient_guess =  quaternion.from_rotation_matrix(a.R)
obj_orient_guess 

### Get it as SE3 (from vacuum grasping skill)

In [45]:
poseSt = PoseStamped()
poseSt.header.frame_id = "hrr_cobot.base_link"
poseSt.pose.position = hrr_common.ros_utils.np2vec3(obj_pos_guess)
poseSt.pose.orientation = hrr_common.ros_utils.np2quat(obj_orient_guess)
object_center = get_SE3_from_pose_stamped(poseSt)
object_center

  -0.9023   -0.1725   -0.3951    0.5       
   0.3775    0.1262   -0.9174    0         
   0.2081   -0.9769   -0.04877   0.45      
   0         0         0         1         


### First column (x-Axis of the object) is the normal vector of the surface, it should point upwards

In [46]:
nvec = object_center.A[:3, 0]
nvec

array([-0.90231954,  0.37750897,  0.20810197])

The end effector will point in the opposite direction of the normal vector. All that is left is how to turn the last link

### The second or third column (y or z-Axis of the object) will match the direction from cup to cup
The following computes end effector pose such that the middle of the vacuum is at the object_center. The y-axis of the object is used to set orientation of the last link.

In [50]:
grasp_pose_EE = hrr_common.utils.calc_goal_pose(nvec, object_center.t, y_axis=-object_center.A[:3, 1])
ee2tip = np.r_[0, 0, 0.20018]
grasp_pose = grasp_pose_EE  @ sm.SE3(-ee2tip)
grasp_pose

   0.3951    0.1725    0.9023    0.3194    
   0.9174   -0.1262   -0.3775    0.07557   
   0.04877   0.9769   -0.2081    0.4917    
   0         0         0         1         


In [14]:
cobot.goTo(grasp_pose, v_max=0.01)

In [47]:
grasp_pose_EE = hrr_common.utils.calc_goal_pose(nvec, object_center.t, y_axis=-object_center.A[:3, 2])
ee2tip = np.r_[0, 0, 0.20018]
grasp_pose = grasp_pose_EE  @ sm.SE3(-ee2tip)
grasp_pose

  -0.1725    0.3951    0.9023    0.3194    
   0.1262    0.9174   -0.3775    0.07557   
  -0.9769    0.04877  -0.2081    0.4917    
   0         0         0         1         


In [48]:
cobot.goTo(grasp_pose, v_max=0.01)

[rosout] joint(s) [3] are/is critical.
[rosout] joint(s) [3] are/is critical.
[rosout] joint(s) [3] are/is critical.
[rosout] joint(s) [3] are/is critical.
[rosout] joint(s) [3] are/is critical.
[rosout] joint(s) [3] are/is critical.
[rosout] joint(s) [3] are/is critical.
[rosout] joint(s) [3] are/is critical.
[rosout] joint(s) [3] are/is critical.
[rosout] joint(s) [3] are/is critical.
[rosout] joint(s) [3] are/is critical.
[rosout] joint(s) [3] are/is critical.
[rosout] joint(s) [3] are/is critical.
[rosout] joint(s) [3] are/is critical.
[rosout] joint(s) [3] are/is critical.
[rosout] joint(s) [3] are/is critical.
[rosout] joint(s) [3] are/is critical.
[rosout] joint(s) [3] are/is critical.
[rosout] joint(s) [3] are/is critical.
[rosout] joint(s) [3] are/is critical.
[rosout] joint(s) [3] are/is critical.
[rosout] joint(s) [3] are/is critical.
[rosout] joint(s) [3] are/is critical.
[rosout] joint(s) [3] are/is critical.
[rosout] joint(s) [3] are/is critical.
[rosout] joint(s) [3] are

In [ ]:
#Compare
cobot.FK(cobot.q_calib), cobot.T_B_E_robot

In [ ]:
cobot.FK(cobot.q_calib), cobot.FK(cobot.q_calib) @ sm.SE3.Rz(np.deg2rad(90)), cobot.FK(cobot.q_calib) @ sm.SE3.Rx(np.deg2rad(90))

### Test

In [13]:
rospy.init_node('Orientation_tester_notebook')
import hrr_cobot_robot as hrr_rob
cobot = hrr_rob.HrrCobotControl.from_ros("/hrr_cobot")

[rosout] [/Orientation_tester_notebook] Initialize MATLAB-Motion planner @ /home/hrr_cobot/_ros/hr_recycler_ws/src/planner/src
[rosout] [/Orientation_tester_notebook] connected to Matlab
Shutting down global node /matlab_global_node_08837 with NodeURI http://192.168.2.87:43353/
The value of the ROS_IP environment variable, 192.168.2.87, will be used to set the advertised address for the ROS node.
Initializing global node /matlab_global_node_28182 with NodeURI http://192.168.2.87:40551/
[rosout] [/Orientation_tester_notebook] MATLAB-Initialization done


pybullet build time: May  8 2021 05:48:13


[rosout] some functions are not yet compiled. Expect delays upon first call


In [51]:
cobot.move_to_joint_pose(cobot.q_calib, stochastic=False)

In [ ]:
cobot.goTo(grasp_pose, v_max=0.01)

In [ ]:
cobot.goTo(cobot.FK(cobot.q_calib) @ sm.SE3.Rx(np.deg2rad(90)),v_max=0.01)

### Tell Dimitris to adjust orientations appropriately
In our lab we had object y-axis (or z-axis, whatever aligns with the cups) always parallel to base y-axis, except for middle cover where we turn 90 degrees

In [ ]:
#To test turning 90 degrees, rotate object frame quaternions around x-axis (which is the one pointing up)
obj_orient_guess = quaternion.from_rotation_matrix(quaternion.as_rotation_matrix(obj_orient_guess) @ sm.SE3.Rx(np.deg2rad(90)).R)
obj_orient_guess

In [ ]:
a, a @ sm.SE3.Rx(np.deg2rad(90)), a @ sm.SE3.Rx(np.deg2rad(-90))